In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as datasets

In [ ]:
# Mean and std of the pixels in the dataset
mean_gray = 0.1307
stddev_gray = 0.3081

# Transform the images to tensors and normalize
transforms = transforms.Compose([transforms.ToTensor(),
                                 transforms.Normalize((mean_gray,),
                                 (stddev_gray,))])

# Load our datasets
train_dataset = datasets.MNIST(root = './data',
                               train = True,
                               transform = transforms)

test_dataset = datasets.MNIST(root = './data',
                               train = False,
                               transform = transforms)

In [ ]:
import matplotlib.pyplot as plt
random_image = train_dataset[20][0].numpy() * stddev_gray + mean_gray
plt.imshow(random_image.reshape(28, 28), cmap="gray")

In [ ]:
print(train_dataset[20][1])

In [ ]:
batch_size = 100

train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                         batch_size = batch_size,
                                         shuffle = True)

test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                         batch_size = batch_size,
                                         shuffle = False)

In [ ]:
print('There are {} images in the training set'.format(len(train_dataset)))
print('There are {} images in the test set'.format(len(test_dataset)))
print('There are {} batches in the train loader'.format(len(train_loader)))
print('There are {} batches in the testloader'.format(len(test_loader)))

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        # First CNN Layer
        self.cnn1 = nn.Conv2d(in_channels=1, out_channels=8, kernel_size=3, stride=1, padding=1) # output 28x28
        self.batchnorm1 = nn.BatchNorm2d(8) # Apply to each feature map (out_channels)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=2) # output 14x14

        # Second CNN Layer
        self.cnn2 = nn.Conv2d(in_channels=8, out_channels=32, kernel_size=5, stride=1, padding=2)
        self.batchnorm2 = nn.BatchNorm2d(32)

        # Fully Connected Layer
        self.fc1 = nn.Linear(in_features=1568, out_features=600) # 7*7*32 = 1568
        self.dropout = nn.Dropout(p=0.5)
        self.fc2 = nn.Linear(in_features=600, out_features=10)

    def forward(self, x):
        # First CNN Layer
        out = self.cnn1(x)
        out = self.batchnorm1(out)
        out = self.relu(out)
        out = self.maxpool(out)

        # Second CNN Layer
        out = self.cnn2(out)
        out = self.batchnorm2(out)
        out = self.relu(out)
        out = self.maxpool(out)

        # Flatten the outputs
        out = out.view(-1, 1568)

        # Fully Connected Layer
        out = self.fc1(out)
        out = self.relu(out)
        out = self.dropout(out)
        out = self.fc2(out)

        return out

In [ ]:
model = CNN()

CUDA = torch.cuda.is_available()
if CUDA:
    model = model.cuda()

loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.001)

In [ ]:
# Understand what's happening
iteration = 0
correct = 0

for i,(inputs,labels) in enumerate (train_loader):

    if CUDA:
        inputs = inputs.cuda()
        labels = labels.cuda()
        
    print("For one iteration, this is what happens:")
    print("Input Shape: ",inputs.shape)
    print("Labels Shape: ",labels.shape)
    output = model(inputs)
    print("Outputs Shape: ",output.shape)
    _, predicted = torch.max(output, 1)
    print("Predicted Shape: ",predicted.shape)
    print("Predicted Tensor: ")
    print(predicted)
    correct += (predicted == labels).sum()
    break

In [ ]:
# Training the CNN
epochs = 10
train_loss = []
train_accuracy = []
test_loss = []
test_accuracy = []

for epoch in range(epochs):
    correct = 0
    iterations = 0
    iter_loss = 0.0

    model.train() # Need this since we're using dropout

    for i, (inputs, labels) in enumerate(train_loader):

        if CUDA:
            inputs = inputs.cuda()
            labels = labels.cuda()
        
        # Forward Prop
        outputs = model(inputs)         

        # Calculate Loss
        loss = loss_function(outputs, labels)  
        iter_loss += loss.item()

        # Backprop
        optimizer.zero_grad() # Clear the gradients
        loss.backward() # Calculates the gradients
        optimizer.step() # Updates the weights

        # Record the correct predictions for training data
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum()
        iterations += 1

    # Record the training loss and training accuracy
    train_loss.append(iter_loss / iterations)
    train_accuracy.append(100 * correct / len(train_dataset))

    # Testing
    correct = 0
    iterations = 0
    testing_loss = 0.0

    model.eval()         

    for i, (inputs, labels) in enumerate(test_loader):

        if CUDA:
            inputs = inputs.cuda()
            labels = labels.cuda()
        
        # Forward Prop
        outputs = model(inputs)

        # Calculate Loss
        loss = loss_function(outputs, labels)
        testing_loss += loss.item()

        # Record the correct predictions for training data
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum()
        iterations += 1
    
    # Record the testing loss and testing accuracy
    test_loss.append(testing_loss / iterations)
    test_accuracy.append(100 * correct / len(test_dataset))

    print ('Epoch {}/{}, Training Loss: {:.3f}, Training Accuracy: {:.3f}, Testing Loss: {:.3f}, Testing Acc: {:.3f}'
           .format(epoch+1, epochs, train_loss[-1], train_accuracy[-1], 
             test_loss[-1], test_accuracy[-1]))

In [ ]:
print(test_loss)

In [ ]:
# Loss
f = plt.figure(figsize=(10, 10))
plt.plot(train_loss, label="Training Loss")
plt.plot(test_loss, label="Test Loss")
plt.legend()
plt.show()

In [ ]:
# Accuracy
f = plt.figure(figsize=(10, 10))
plt.plot(train_accuracy, label="Training Accuracy")
plt.plot(test_accuracy, label="Test Accuracy")
plt.legend()
plt.show()

In [ ]:
img = test_dataset[30][0].resize_((1, 1, 28, 28)) # (batch_size, channels, height, width)
label = test_dataset[30][1]

if CUDA:
    model = model.cuda()
    img = img.cuda()

output = model(img)
_, predicted = torch.max(output, 1)
print(f"Prediction is: {predicted.item()}")
print(f"Actual is: {label}")